In [2]:
#Dependencies
import pandas as pd
import numpy as np

#load file
file = "Resources/purchase_data.csv"

#read purchase file
purchase_data = pd.read_csv(file)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [81]:
#Calculate total number of players
players_sum = purchase_data["SN"].nunique()
print("Total Players:")
print(players_sum)

#Number of unique items, use item ID
unique_items = purchase_data["Item ID"].nunique()

#Average price calculation
avg_price = purchase_data["Price"].mean()

#Number of purchases made
purchase_count = purchase_data["Purchase ID"].count()

#Calculate total revenue
total_revenue = purchase_data["Price"].sum()

Total Players:
576


In [4]:
#Create analysis table
analysis_df = pd.DataFrame({"Number of Unique Items": [unique_items], "Average Price": [avg_price],
                           "Number of Purchases":[purchase_count], "Total Revenue":[total_revenue]})

#Format currency values
analysis_df["Average Price"] = analysis_df["Average Price"].map("${:,.2f}".format)
analysis_df["Total Revenue"] = analysis_df["Total Revenue"].map("${:,.2f}".format)

analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [5]:
#Remove any duplicate players
no_dup_id = purchase_data.drop_duplicates(["SN"], keep = "last")

#Find percentage and count of male players
male_count = len(no_dup_id[no_dup_id['Gender'] == 'Male'])
male_percent = male_count/players_sum
#print(male_count)
#print(male_percent)

#Find percentage and count of female players
female_count = len(no_dup_id[no_dup_id['Gender'] == 'Female'])
female_percent = female_count/players_sum
#print(female_count)
#print(female_percent)

#Find percentage and count of other/undisclosed
other_count = players_sum - male_count - female_count
other_percent = other_count/players_sum
#print(other_count)
#print(other_percent)

#Create dataframe for gender demographics
gender_demo_df = pd.DataFrame({"Gender":["Male", "Female", "Other/Non-Disclosed"],
                              "Total Count":[male_count, female_count, other_count],
                              "Percentage of Players":[male_percent, female_percent, other_percent]})
gender_demo_df["Percentage of Players"] = pd.Series(["{0:.2f}%".format(val*100) for val in gender_demo_df["Percentage of Players"]],
                                                    index=gender_demo_df.index)
gender_demo_df.set_index(["Gender"], inplace=True)
del gender_demo_df.index.name

gender_demo_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


In [6]:
#Purchasing analysis based on gender
#demo numbers
gender_num = purchase_data.groupby("Gender").SN.count()
gender_num

Gender
Female                   113
Male                     652
Other / Non-Disclosed     15
Name: SN, dtype: int64

In [14]:
#acquire number of purchases by gender
purchase_count_by_gen= pd.DataFrame(purchase_data.groupby("Gender")["Gender"].count())

#sum price according to gender
tot_purchase_by_gen = pd.DataFrame(purchase_data.groupby("Gender")["Price"].sum())

#Merge previous two dataframes
purchase_analysis = pd.merge(purchase_count_by_gen, tot_purchase_by_gen, left_index=True, right_index=True)

#Rename columns
purchase_analysis.rename(columns = {"Gender":"Purchase Count", "Price":"Total Purchase Price"}, inplace=True)

#Calculate avg purchase price by gender and add to new column
purchase_analysis["Average Purchase Price"] = purchase_analysis["Total Purchase Price"]/purchase_analysis["Purchase Count"]

#Add count of each gender to the dataframe
purchase_analysis["Total Count"] = [81, 484, 11]

#Calculate average total purchase per person
purchase_analysis["Avg Total Purchases per Person"] = purchase_analysis["Total Purchase Price"]/purchase_analysis["Total Count"]

#Format appropriate columns as currency
#purchase_analysis.style.format({'Total Purchase Price': '${:.2f}', 'Average Purchase Price': '${:.2f}', 'Avg Total Purchases per Person': '${:.2f}'})
purchase_analysis["Total Purchase Price"] = pd.Series(["${0:.2f}".format(val) for val in purchase_analysis["Total Purchase Price"]],
                                                    index=purchase_analysis.index)
purchase_analysis["Average Purchase Price"] = pd.Series(["${0:.2f}".format(val) for val in purchase_analysis["Average Purchase Price"]],
                                                    index=purchase_analysis.index)
purchase_analysis["Avg Total Purchases per Person"] = pd.Series(["${0:.2f}".format(val) for val in purchase_analysis["Avg Total Purchases per Person"]],
                                                    index=purchase_analysis.index)
del purchase_analysis.index.name

purchase_analysis

,Purchase Count,Total Purchase Price,Average Purchase Price,Total Count,Avg Total Purchases per Person
Female,113,$361.94,$3.20,81,$4.47
Male,652,$1967.64,$3.02,484,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,11,$4.56


In [26]:
#Analysis based on age demographics

#Establish bins for ages
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]

#Label bins
bin_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Get screen names and ages from original table
age_df = purchase_data.iloc[:,[1,2]]

#Remove duplicate SNs
age_df = age_df.drop_duplicates(subset=["SN"], keep="first", inplace=False)

#Add new column with corresponding age range
age_df["Age Range"] = pd.cut(age_df["Age"], bins, labels=bin_labels)

#Sort values from lowest age to greatest
age_df.sort_values(by=["Age"], ascending=True, inplace=True)

#Create new dataframe with just the age ranges and number of occurences
age_group_df = age_df.groupby(["Age Range"]).size().to_frame(name="Total Count").reset_index()

#Calculate percentage of players by age group
total_players = age_group_df["Total Count"].sum()
age_group_df["Percentage of Players"] = (age_group_df["Total Count"]/total_players)
age_group_df["Percentage of Players"] = pd.Series(["{0:.2f}%".format(val*100) for val in age_group_df["Percentage of Players"]],
                                                    index=age_group_df.index)
age_group_df.set_index(["Age Range"], inplace=True)
del age_group_df.index.name

age_group_df.head()

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%


In [53]:
#ANALYZE PURCHASES BY AGE

#Add age groups to initial dataframe
purchase_data["Age Group"] = pd.cut(purchase_data["Age"], bins, labels=bin_labels)

#Use Groupby for calculations
purchase_groups = purchase_data.groupby("Age Group")

#Total number of purchases by age
num_purchase_age = purchase_groups["Price"].count()
#print(num_purchase_age)

#Average purchase price by age
avg_price_age = purchase_groups["Price"].mean()
#print(avg_price_age)

#Total purchase value by age
total_purchase_age = purchase_groups["Price"].sum()
#print(total_purchase_age)

#Avg total purchase by age
unique_user = purchase_groups["SN"].nunique()
avg_purchase_age = total_purchase_age/unique_user
#print(avg_purchase_age)

#Create Dataframe
purchase_age_df = pd.DataFrame({"Purchase Count": num_purchase_age, "Average Purchase Price": avg_price_age,
                               "Total Purchase Value": total_purchase_age, "Avg Total Purchases per Person": avg_purchase_age})
purchase_age_df["Total Purchase Value"] = pd.Series(["${0:.2f}".format(val) for val in purchase_age_df["Total Purchase Value"]],
                                                    index=purchase_age_df.index)
purchase_age_df["Average Purchase Price"] = pd.Series(["${0:.2f}".format(val) for val in purchase_age_df["Average Purchase Price"]],
                                                    index=purchase_age_df.index)
purchase_age_df["Avg Total Purchases per Person"] = pd.Series(["${0:.2f}".format(val) for val in purchase_age_df["Avg Total Purchases per Person"]],
                                                    index=purchase_age_df.index)
del purchase_age_df.index.name
purchase_age_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchases per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [62]:
#TOP SPENDERS

#Calculate based on unique SNs
#Total purchases per person
purchase_tot_sn = purchase_data.groupby("SN")["Price"].sum()
#print(purchase_tot_sn)

#Total number of purchases per person
purchase_num_sn = purchase_data.groupby("SN")["Price"].count()
#print(purchase_num_sn)

#Average purchase value per person
purchase_avg_sn = purchase_data.groupby("SN")["Price"].mean()
#print(purchase_avg_sn)

#Add values to new dataframe
purchase_sn_df = pd.DataFrame({"Purchase Count":purchase_num_sn, "Average Purchase Price":purchase_avg_sn,
                           "Total Purchase Value":purchase_tot_sn})

#Sort from highest total purchase to lowest
purchase_sn_df.sort_values("Total Purchase Value", ascending=False, inplace=True)

purchase_sn_df["Total Purchase Value"] = pd.Series(["${0:.2f}".format(val) for val in purchase_sn_df["Total Purchase Value"]],
                                                    index=purchase_sn_df.index)
purchase_sn_df["Average Purchase Price"] = pd.Series(["${0:.2f}".format(val) for val in purchase_sn_df["Average Purchase Price"]],
                                                    index=purchase_sn_df.index)

purchase_sn_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [71]:
#MOST POPULAR ITEMS

#Retrieve item ID, item name, and item price columns

#Item count
item_count = purchase_data.groupby(["Item ID", "Item Name"])["Price"].count()
#print(item_count)

#Total purchase for each item
item_total = purchase_data.groupby(["Item ID", "Item Name"])["Price"].sum()
#print(item_total)

#Item price
item_price = purchase_data.groupby(["Item ID", "Item Name"])["Price"].mean()
#print(item_price)

#Create Dataframe
items_df = pd.DataFrame({"Purchase Count":item_count, "Item Price":item_price, "Total Purchase Value":item_total})

#Sort by purchase count
items_sorted = items_df.sort_values("Purchase Count", ascending=False)

items_sorted["Total Purchase Value"] = pd.Series(["${0:.2f}".format(val) for val in items_sorted["Total Purchase Value"]],
                                                    index=items_sorted.index)
items_sorted["Item Price"] = pd.Series(["${0:.2f}".format(val) for val in items_sorted["Item Price"]],
                                                    index=items_sorted.index)

items_sorted.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [76]:
#MOST PROFITABLE ITEMS

#Sort previous able by total purchase value in descending order
items_revenue = items_df.sort_values("Total Purchase Value", ascending=False)
items_revenue["Total Purchase Value"] = pd.Series(["${0:.2f}".format(val) for val in items_revenue["Total Purchase Value"]],
                                                    index=items_revenue.index)
items_revenue["Item Price"] = pd.Series(["${0:.2f}".format(val) for val in items_revenue["Item Price"]],
                                                    index=items_revenue.index)
items_revenue.head(10)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
78,"Glimmer, Ender of the Moon",7,$4.40,$30.80
72,Winter's Bite,8,$3.77,$30.16
